In [1]:
import numpy as np
from utils import *
from utils import *
from dgp import sample_network_chain, get_graph, sample_Y1, agcEffect
import numpy as np


def prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree):
    # 1. Simulate or load data
    adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
    print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())

    tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
    rho = np.array([[0,0.1,0.2],
                    [0.1,0,0.1],
                    [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
    nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
    gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
    beta = np.array([-1*min_degree,2,-0.2,2,0.1,-2,0.1,2,0.1,0.1])  # shape (10,)

    res_truth1 = agcEffect(
        adj1,
        tau, rho, nu, beta,
        treatment_allocation=0.7,
        R=100,
        burnin_R=100,
        seed=0
    )
    res_truth2 = agcEffect(
        adj1,
        tau, rho, nu, beta,
        treatment_allocation=0.3,
        R=100,
        burnin_R=100,
        seed=0
    )

    ground_truth1 = {}
    ground_truth1['average'] = res_truth1['average']
    ground_truth1['direct'] = res_truth1['direct_effect']
    ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
    ground_truth1['spillover_effect'] = res_truth1['spillover_effect']
    ground_truth1['psi_1_gamma'] = res_truth1['psi_1_gamma']
    ground_truth1['psi_0_gamma'] = res_truth1['psi_0_gamma']
    ground_truth1['psi_zero'] = res_truth1['psi_zero']

    return adj1, None, None, None, ground_truth1

num_sample = 300
num_burn = 20
n_cpu = 10

In [28]:
sample_size = 800
min_degree = 1
max_degree = 2

adj1, Y_chain, A_chain, L_chain, ground_truth1 = prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)

1 2


100%|██████████| 200/200 [00:00<00:00, 585.79it/s]


In [29]:
results1 = np.load(f'run/sim_results_v1/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_None.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results_v1/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_None.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 2000/2000 [01:17<00:00, 25.80it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.9365, 0.9465, 0.9225, 0.923 , 0.939 , 0.941 , 0.926 ]),
 'bias': array([-3.97170239e-06, -4.97438723e-03, -1.02080362e-03, -1.65494237e-03,
        -3.45003787e-03,  1.52434936e-03,  3.17929173e-03]),
 'mse': array([0.00255494, 0.0071886 , 0.00254044, 0.00691059, 0.00462743,
        0.00276393, 0.00453894]),
 'var': array([0.00255494, 0.00716386, 0.00253939, 0.00690785, 0.00461553,
        0.0027616 , 0.00452883]),
 'ci_length': array([0.17748107, 0.30267735, 0.16724886, 0.27947816, 0.23447915,
        0.18448109, 0.22786862]),
 'true_effect': array([ 0.59675  ,  0.3241125, -0.0245   , -0.0437125,  0.6959375,
         0.371825 ,  0.4155375])}

In [30]:
results1 = np.load(f'run/sim_results_v1/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_outcome.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results_v1/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_outcome.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 2000/2000 [01:30<00:00, 22.03it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.888 , 0.861 , 0.934 , 0.951 , 0.8385, 0.883 , 0.8935]),
 'bias': array([ 0.00987225,  0.03091379, -0.005285  , -0.00281174,  0.01719264,
        -0.01372115, -0.01090941]),
 'mse': array([0.00389976, 0.01221377, 0.00425376, 0.01170596, 0.007111  ,
        0.00484066, 0.00800087]),
 'var': array([0.0038023 , 0.01125811, 0.00422583, 0.01169805, 0.00681541,
        0.00465239, 0.00788185]),
 'ci_length': array([0.24084966, 0.43304659, 0.24688292, 0.42537292, 0.31922124,
        0.2634953 , 0.3495642 ]),
 'true_effect': array([ 0.59675  ,  0.3241125, -0.0245   , -0.0437125,  0.6959375,
         0.371825 ,  0.4155375])}

In [31]:
results1 = np.load(f'run/sim_results_v1/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_None.npy')
results2 = np.load(f'run/sim_results_v1/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_None.npy')

compute_stats_ag(results1, results2, ground_truth1)

{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 0.00478266, -0.00238793,  0.0005349 , -0.00201616,  0.00179144,
         0.00417936,  0.00619553]),
 'mse': array([0.00117209, 0.00247958, 0.00060197, 0.00222894, 0.00164926,
        0.00174902, 0.00173701]),
 'var': array([0.00114921, 0.00247388, 0.00060168, 0.00222487, 0.00164605,
        0.00173156, 0.00169863]),
 'ground_truth': array([ 0.59675  ,  0.3241125, -0.0245   , -0.0437125])}

In [32]:
results1 = np.load(f'run/sim_results_v1/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_outcome.npy')
results2 = np.load(f'run/sim_results_v1/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_outcome.npy')

compute_stats_ag(results1, results2, ground_truth1)

{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 0.07595224,  0.30178171,  0.01216311,  0.00637985,  0.16439199,
        -0.13738973, -0.14376957]),
 'mse': array([0.00611011, 0.09399936, 0.00043838, 0.00249761, 0.02759847,
        0.02071189, 0.02150087]),
 'var': array([0.00034137, 0.00292715, 0.00029044, 0.00245691, 0.00057374,
        0.00183595, 0.00083118]),
 'ground_truth': array([ 0.59675  ,  0.3241125, -0.0245   , -0.0437125])}

'bias': array([ 0.00987225,  0.03091379, -0.005285  , -0.00281174,  0.01719264,
        -0.01372115, -0.01090941]),
 'mse': array([0.00389976, 0.01221377, 0.00425376, 0.01170596, 0.007111  ,
        0.00484066, 0.00800087]),

In [7]:
sample_size = 800
min_degree = 2
max_degree = 5

adj1, Y_chain, A_chain, L_chain, ground_truth1 = prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)

2 5


100%|██████████| 200/200 [00:00<00:00, 578.56it/s]


In [8]:
results1 = np.load(f'run/sim_results_v1/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_None.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results_v1/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_None.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 2000/2000 [03:03<00:00, 10.92it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.9545, 0.955 , 0.9295, 0.9265, 0.959 , 0.957 , 0.9145]),
 'bias': array([ 0.00191133, -0.0127017 ,  0.00188376,  0.00582053, -0.00505044,
         0.00765127,  0.00183074]),
 'mse': array([0.00263467, 0.00894492, 0.00598617, 0.02831693, 0.00464277,
        0.0042035 , 0.0249686 ]),
 'var': array([0.00263102, 0.00878359, 0.00598262, 0.02828305, 0.00461727,
        0.00414496, 0.02496525]),
 'ci_length': array([0.19840618, 0.35563979, 0.26958793, 0.5502538 , 0.25775937,
        0.23874945, 0.48911147]),
 'true_effect': array([ 0.4657375,  0.3202875, -0.0436625, -0.079675 ,  0.564975 ,
         0.2446875,  0.3243625])}

In [9]:
results1 = np.load(f'run/sim_results_v1/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_outcome.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results_v1/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_outcome.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 2000/2000 [03:05<00:00, 10.77it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.954 , 0.958 , 0.954 , 0.791 , 0.9505, 0.9045, 0.6765]),
 'bias': array([-0.00559319, -0.02810395,  0.02199954,  0.06270418, -0.01717563,
         0.01092832, -0.05177586]),
 'mse': array([0.00431446, 0.01486089, 0.01003254, 0.0483772 , 0.00785068,
        0.0066165 , 0.04123825]),
 'var': array([0.00428317, 0.01407106, 0.00954856, 0.04444538, 0.00755568,
        0.00649707, 0.03855752]),
 'ci_length': array([0.27908471, 0.49872526, 0.36104371, 0.73168999, 0.36880245,
        0.31369855, 0.63752076]),
 'true_effect': array([ 0.4657375,  0.3202875, -0.0436625, -0.079675 ,  0.564975 ,
         0.2446875,  0.3243625])}

In [10]:
results1 = np.load(f'run/sim_results_v1/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_None.npy')
results2 = np.load(f'run/sim_results_v1/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_None.npy')

compute_stats_ag(results1, results2, ground_truth1)

{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 0.00755256, -0.00961854, -0.00525555,  0.00621677,  0.0015515 ,
         0.01117004,  0.00495326]),
 'mse': array([0.00128547, 0.00262876, 0.00191428, 0.00424377, 0.00157566,
        0.00221983, 0.00279822]),
 'var': array([0.00122843, 0.00253624, 0.00188666, 0.00420512, 0.00157325,
        0.00209506, 0.00277369]),
 'ground_truth': array([ 0.4657375,  0.3202875, -0.0436625, -0.079675 ])}

In [11]:
results1 = np.load(f'run/sim_results_v1/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_outcome.npy')
results2 = np.load(f'run/sim_results_v1/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_outcome.npy')

compute_stats_ag(results1, results2, ground_truth1)

{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 0.08962437,  0.30394135,  0.01516175,  0.04749685,  0.18121664,
        -0.12272471, -0.17022156]),
 'mse': array([0.00844072, 0.09584728, 0.00178089, 0.00438369, 0.03393199,
        0.01635197, 0.02981207]),
 'var': array([0.00040819, 0.00346693, 0.00155101, 0.00212773, 0.00109252,
        0.00129061, 0.00083669]),
 'ground_truth': array([ 0.4657375,  0.3202875, -0.0436625, -0.079675 ])}

 'bias': array([-0.00559319, -0.02810395,  0.02199954,  0.06270418, -0.01717563,
         0.01092832, -0.05177586]),
 'mse': array([0.00431446, 0.01486089, 0.01003254, 0.0483772 , 0.00785068,
        0.0066165 , 0.04123825]),

In [19]:
sample_size = 800
min_degree = 3
max_degree = 10

adj1, Y_chain, A_chain, L_chain, ground_truth1 = prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)

3 10


100%|██████████| 200/200 [00:00<00:00, 447.80it/s]


In [13]:
results1 = np.load(f'run/sim_results_v1/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_None.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results_v1/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_None.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 2000/2000 [05:35<00:00,  5.96it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.982 , 0.981 , 0.951 , 0.9305, 0.9765, 0.965 , 0.746 ]),
 'bias': array([ 0.00359751, -0.01876072,  0.00728146,  0.01845073, -0.00683321,
         0.01192751, -0.00652322]),
 'mse': array([0.00165485, 0.00785293, 0.01228292, 0.03800267, 0.00234618,
        0.00550051, 0.03377565]),
 'var': array([0.00164191, 0.00750096, 0.0122299 , 0.03766224, 0.00229949,
        0.00535824, 0.0337331 ]),
 'ci_length': array([0.18125234, 0.37074743, 0.41041514, 0.59285289, 0.21974135,
        0.29389292, 0.47312103]),
 'true_effect': array([ 0.34095  ,  0.27945  , -0.0546625, -0.10395  ,  0.4295875,
         0.1501375,  0.2540875])}

In [20]:
results1 = np.load(f'run/sim_results_v1/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_outcome.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results_v1/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_outcome.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 2000/2000 [05:17<00:00,  6.29it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.967 , 0.977 , 0.8935, 0.387 , 0.956 , 0.924 , 0.31  ]),
 'bias': array([-0.0350118 , -0.11157833,  0.0794579 ,  0.1666767 , -0.0732878 ,
         0.03829053, -0.12838617]),
 'mse': array([0.00407891, 0.02479339, 0.02443653, 0.07822279, 0.00943257,
        0.01014024, 0.05911056]),
 'var': array([0.00285308, 0.01234366, 0.01812297, 0.05044167, 0.00406147,
        0.00867408, 0.04262755]),
 'ci_length': array([0.25564063, 0.50135096, 0.51348122, 0.67700344, 0.32191444,
        0.36458838, 0.48627013]),
 'true_effect': array([ 0.34095  ,  0.27945  , -0.0546625, -0.10395  ,  0.4295875,
         0.1501375,  0.2540875])}

In [21]:
results1 = np.load(f'run/sim_results_v1/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_None.npy')
results2 = np.load(f'run/sim_results_v1/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_None.npy')

compute_stats_ag(results1, results2, ground_truth1)

{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 8.78678750e-03, -1.53919125e-02, -1.53904125e-02,  1.57455750e-02,
         8.34250000e-05,  1.54753375e-02, -2.70237500e-04]),
 'mse': array([0.00097653, 0.00226939, 0.00232105, 0.00379792, 0.00124438,
        0.00165016, 0.00364713]),
 'var': array([0.00089933, 0.00203247, 0.00208418, 0.00355   , 0.00124437,
        0.00141067, 0.00364706]),
 'ground_truth': array([ 0.34095  ,  0.27945  , -0.0546625, -0.10395  ])}

In [22]:
results1 = np.load(f'run/sim_results_v1/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_outcome.npy')
results2 = np.load(f'run/sim_results_v1/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_outcome.npy')

compute_stats_ag(results1, results2, ground_truth1)

{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 0.07871949,  0.24962319,  0.01968974,  0.09147404,  0.15585427,
        -0.09376891, -0.18524295]),
 'mse': array([0.00654387, 0.06496294, 0.00266992, 0.00877628, 0.02590736,
        0.00911135, 0.03465573]),
 'var': array([0.00034712, 0.0026512 , 0.00228224, 0.00040878, 0.00161681,
        0.00031874, 0.00034078]),
 'ground_truth': array([ 0.34095  ,  0.27945  , -0.0546625, -0.10395  ])}

'bias': array([-0.0350118 , -0.11157833,  0.0794579 ,  0.1666767 , -0.0732878 ,
         0.03829053, -0.12838617]),
 'mse': array([0.00407891, 0.02479339, 0.02443653, 0.07822279, 0.00943257,
        0.01014024, 0.05911056]),